# Создаем сессию

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession\
    .builder\
    .appName("Spark handle file and upload to hdfs")\
    .getOrCreate()

# Открываем файл 

In [ ]:
raw = spark\
    .read\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv("otus_test/otus_train.csv")

# Убираем первую и последние две колонки

In [ ]:
columns = raw.columns
columnsLen = len(columns)
colsToDrop = columns[0].split() + columns[columnsLen-2:columnsLen]
df = raw.drop(*colsToDrop)

df.show(5, False)

# Перемножить Total_Trans_Ct на Total_Ct_Chng_Q4_Q1

In [ ]:
df.withColumn('features_1', col('Total_Ct_Chng_Q4_Q1')*col('Total_Trans_Ct'))

# Сложить Avg_Utilization_Ratio с Customer_Age переменными

In [ ]:
df.withColumn('features_2', col('Total_Ct_Chng_Q4_Q1') + col('Total_Trans_Ct'))

# Посчитать признак заданный формулой

 $$F = \frac{(Customer\_Age - Dependent\_count)}{(Total\_Relationship\_Count + Months\_on\_book)}$$

In [ ]:
df.withColumn('features_3', (col('Customer_Age') - col('Dependent_count'))/(col('Total_Relationship') + col('Months_on_book')))

# Отборать Id и новый признаки и загрузить в HDFS

In [ ]:
df.select('features_1','features_2','features_3').write.mode("overwrite").parquet('final_otus_train.csv')